In [1]:
!test -d yolov5 || git clone https://github.com/ultralytics/yolov5
!ls
!test -f tacotrashdataset.zip  && test -d tacotrashdataset || unzip tacotrashdataset.zip  -d tacotrashdataset
!test -f yolov5-taco.zip && test -d yolov5-taco || unzip yolov5-taco.zip -d yolov5-taco
%cd yolov5
!pip install --upgrade pip
!pip install requirements.txt
!pip install pycocotools split-folders wandb torch==1.7.1 tensorboard opencv-python torchvision

Cloning into 'yolov5'...
remote: Enumerating objects: 11371, done.
remote: Total 11371 (delta 0), reused 0 (delta 0), pack-reused 11371
Receiving objects: 100% (11371/11371), 11.28 MiB | 17.64 MiB/s, done.
Resolving deltas: 100% (7844/7844), done.
docker-compose.yaml  run_notebook.sh	   wandb	      yolov5-taco.zip
Dockerfile	     taco		   yolov5
name_map.py	     tacotrashdataset	   yolov5-taco
README.md	     tacotrashdataset.zip  yolov5-taco.ipynb
/home/jovyan/work/yolov5
ERROR: Could not find a version that satisfies the requirement requirements.txt (from versions: none)
HINT: You are attempting to install a package literally named "requirements.txt" (which cannot exist). Consider using the '-r' flag to install the packages listed in requirements.txt
ERROR: No matching distribution found for requirements.txt


In [2]:
from pycocotools.coco import COCO
import numpy as np
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
annotation_path = '../tacotrashdataset/data/annotations.json'

data_source = COCO(annotation_file=annotation_path)
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
categories.sort(key=lambda x: x['id'])

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


### Class dictionary

In [3]:
classes = {}
taco_labels = {}
taco_labels_inverse = {}

for c in categories:
    taco_labels[len(classes)] = c['id']
    taco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)

### Split data for training, validation, testing

In [4]:
!mkdir -p tmp/labels tmp/images
!ls

CONTRIBUTING.md  export.py   README.md	       train.py
data		 hubconf.py  requirements.txt  tutorial.ipynb
detect.py	 LICENSE     setup.cfg	       utils
Dockerfile	 models      tmp	       val.py


In [5]:
IMAGES_PATH = 'tmp/images/'
LABELS_PATH = 'tmp/labels/'
import shutil
import os
img_ids = data_source.getImgIds()

for index, img_id in tqdm.tqdm(enumerate(img_ids)):
    img_info = data_source.loadImgs(img_id)[0]
    # img_dir: batch_x/.....jpg ---> batch_x_......jpg
    img_dir = img_info['file_name'].replace('/', '_')
    
    image_name = img_dir.split('.')[0]
    label_dir = LABELS_PATH + image_name + '.txt'
    
    height = img_info['height']
    width = img_info['width']

    # get images
    shutil.copy(f"../tacotrashdataset/data/{img_info['file_name']}", os.path.join(IMAGES_PATH, img_dir))

    # get labels
    with open(label_dir, mode='w') as fp:
        annotation_id = data_source.getAnnIds(img_id)
        if len(annotation_id) == 0:
            fp.write('')
            continue
        boxes = np.zeros((0, 5))
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        for annotation in annotations:
            label = taco_labels_inverse[annotation['category_id']]
            box = annotation['bbox']
            # some annotations have basically no width / height (extremely small), skip them
            if box[2] < 1 or box[3] < 1:
                continue
            # top_x,top_y,width,height ----> cen_x,cen_y,width,height
            # standardize to 0-1
            box[0] = round((box[0] + box[2] / 2) / width, 6)
            box[1] = round((box[1] + box[3] / 2) / height, 6)
            box[2] = round(box[2] / width, 6)
            box[3] = round(box[3] / height, 6)
            # line: label x_center y_center width height
            lines = lines + str(label)
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        fp.writelines(lines)

1500it [00:01, 969.18it/s] 


In [6]:
print(len(os.listdir(IMAGES_PATH)))
print(len(os.listdir(LABELS_PATH)))

1500
1500


In [7]:
import splitfolders
splitfolders.ratio('tmp', output='taco', seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 3000 files [00:01, 1917.06 files/s]


Check files:

In [8]:
print(sorted(os.listdir('taco/train/images'))[:5])
print(sorted(os.listdir('taco/val/images'))[:5])
print(sorted(os.listdir('taco/test/images'))[:5])

['batch_10_000001.jpg', 'batch_10_000002.jpg', 'batch_10_000003.jpg', 'batch_10_000004.jpg', 'batch_10_000006.jpg']
['batch_10_000000.jpg', 'batch_10_000005.jpg', 'batch_10_000014.jpg', 'batch_10_000026.jpg', 'batch_10_000031.jpg']
['batch_10_000025.jpg', 'batch_10_000035.jpg', 'batch_10_000036.jpg', 'batch_10_000050.jpg', 'batch_10_000087.jpg']


Remove tmp fodler

In [9]:
!rm -r tmp

### Training

Run this if the error "a view of a leaf Variable that requires grad is being used in an in-place operation", then train again

In [ ]:
# Be forewanred, this takes ~6 hours on 16 i9 cores
import time
start = time.time()
!wandb disabled
!python train.py --img 640 --batch 64 --epochs 50 --data ../yolov5-taco/taco.yaml --weights ../yolov5-taco/50_epochs_trained_last.pt
end = time.time()

In [ ]:
print('Training time:', end - start, 'seconds')

### Show the training result

In [ ]:
%cd 'runs/train/exp4'
%ls

In [ ]:
res_path = 'results.png'
img = plt.imread(res_path)
plt.figure(figsize=(20, 20))
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.show()

### Detect

In [ ]:
%cd /home/jovyan/work/yolov5
!ls

In [ ]:
!wandb disabled
!python detect.py --weights runs/train/exp4/weights/best.pt --img 640 --conf 0.25 --source taco/test/images 

In [ ]:
OUT_PATH = 'runs/detect/exp2'
cnt = 0
for file in os.listdir(OUT_PATH):
    img = plt.imread(os.path.join(OUT_PATH, file))
    plt.figure(figsize=(15, 15))
    plt.imshow(img)
    if cnt == 15:
        break
    plt.xticks([])
    plt.yticks([])
    plt.show()
    cnt += 1

In [ ]:
!python export.py --weights runs/train/exp4/weights/best.pt --include tfjs 